In [2]:
import datafaucet as dfc

In [3]:
# start the engine
project = dfc.project.load()

created SparkEngine
Init engine "spark"
Configuring packages:
  -  mysql:mysql-connector-java:8.0.12
  -  org.apache.hadoop:hadoop-aws:3.2.1
Configuring conf:
  -  spark.hadoop.fs.s3a.access.key : ****** (redacted)
  -  spark.hadoop.fs.s3a.endpoint : http://minio:9000
  -  spark.hadoop.fs.s3a.impl : org.apache.hadoop.fs.s3a.S3AFileSystem
  -  spark.hadoop.fs.s3a.path.style.access : true
  -  spark.hadoop.fs.s3a.secret.key : ****** (redacted)
Connecting to spark master: local[*]
Engine context spark:2.4.4 successfully started


In [7]:
spark = dfc.context()

In [8]:
df = spark.range(100) 

In [9]:
df.data.grid()

AttributeError: 'DataFrame' object has no attribute 'data'

In [5]:
(df
     .cols.get('name').obscure(alias='enc')
     .cols.get('enc').unravel(alias='dec')
).data.grid()

,id,name,trade,fight,greedy,enc,dec
0,0,human,0.3,0.7,0.5,WkBfVFw=,human
1,1,dwarf,0.6,0.4,0.8,VkJTR1Q=,dwarf
2,2,orc,0.2,0.8,0.5,Vkta,orc
3,3,elf,0.8,0.2,0.2,XFVf,elf


In [7]:
df.data.grid().groupby(['id', 'name'])\
  .agg({'fight':[max, 'min'], 'trade': 'count'}).stack(0)

count  max  min
id name                        
0  human fight    NaN  0.7  0.7
         trade    1.0  NaN  NaN
1  dwarf fight    NaN  0.4  0.4
         trade    1.0  NaN  NaN
2  orc   fight    NaN  0.8  0.8
         trade    1.0  NaN  NaN
3  elf   fight    NaN  0.2  0.2
         trade    1.0  NaN  NaN

In [18]:
from pyspark.sql import functions as F

In [23]:
df.cols.get('groupby('id', 'name')\
  .agg({'fight':[F.max, 'min'], 'trade': 'min'}).data.grid()

,name,id,colname,min,count
0,dwarf,1,fight,0.4,1
1,human,0,fight,0.7,1
2,elf,3,fight,0.2,1
3,orc,2,fight,0.8,1
4,dwarf,1,trade,0.6,1
5,human,0,trade,0.3,1
6,elf,3,trade,0.8,1
7,orc,2,trade,0.2,1
8,dwarf,1,greedy,0.8,1
9,human,0,greedy,0.5,1


INFO:kafka.client:Closing idle connection 1001, last active 36559151 ms ago


In [29]:
df.cols.groupby('id', 'name')\
    .agg({'fight':[F.max, 'min'], 'trade': 'count'}, stack=True).data.grid()

,name,id,colname,max,min,count
0,elf,3,fight,0.2,0.2,NaN
1,orc,2,fight,0.8,0.8,NaN
2,dwarf,1,fight,0.4,0.4,NaN
3,human,0,fight,0.7,0.7,NaN
4,human,0,trade,NaN,NaN,1.0
5,elf,3,trade,NaN,NaN,1.0
6,orc,2,trade,NaN,NaN,1.0
7,dwarf,1,trade,NaN,NaN,1.0


In [28]:
from pyspark.sql import functions as F
df.groupby('id', 'name').agg(
    F.lit('fight').alias('colname'), 
    F.min('fight').alias('min'),
    F.max('fight').alias('max'),
    F.lit(None).alias('count')).union(
    df.groupby('id', 'name').agg(
    F.lit('trade').alias('colname'), 
    F.lit(None).alias('min'),
    F.lit(None).alias('max'),
    F.count('trade').alias('count'))
).data.grid()


,id,name,colname,min,max,count
0,1,dwarf,fight,0.4,0.4,NaN
1,2,orc,fight,0.8,0.8,NaN
2,0,human,fight,0.7,0.7,NaN
3,3,elf,fight,0.2,0.2,NaN
4,1,dwarf,trade,NaN,NaN,1.0
5,2,orc,trade,NaN,NaN,1.0
6,0,human,trade,NaN,NaN,1.0
7,3,elf,trade,NaN,NaN,1.0


In [10]:
        def string2func(func):
            if isinstance(func, str):
                f = A.all.get(func)
                if f:
                    return (func,f)
                else:
                    raise ValueError(f'function {func} not found')
            elif isinstance(func, (type(lambda x: x), type(max))):
                return (func.__name__, func)
            else:
                raise ValueError('Invalid aggregation function')
            
        def parse_single_func(func):
            if isinstance(func, (str, type(lambda x: x), type(max))):
                return string2func(func)
            elif isinstance(func, (tuple)):
                if len(func)==2:
                    return (func[0], string2func(func[1])[1])
                else:
                    raise ValueError('Invalid list/tuple')
            else:
                raise ValueError(f'Invalid aggregation item {func}')
        
        def parse_list_func(func):
            func = [func] if type(func)!=list else func
            return [parse_single_func(x) for x in func]

        def parse_dict_func(func):
            func = {0: func} if not isinstance(func, dict) else func
            return {x[0]:parse_list_func(x[1]) for x in func.items()}


In [14]:
lst = [
    F.max,
    'max',
    ('maxx', F.max),
    ('maxx', 'max'),
    ['max', F.max, ('maxx', F.max)],
    {'a': F.max},
    {'a': 'max'},
    {'a': ('maxx', F.max)},
    {'a': ('maxx', 'max')},
    {'a': ['max', F.max, ('maxx', F.max)]},
    {'a': F.max, 'b': F.max},
    {'a': 'max', 'b': 'max'},
    {'a': ('maxx', F.max), 'b': ('maxx', F.max)},
    {'a': ('maxx', 'max'), 'b': ('maxx', 'max')},
    {'a': ['max', F.max, ('maxx', F.max)], 'b': ['min', F.min, ('minn', F.min)]}
]
for i in lst:    
    print('=====')
    print(i)
    funcs = parse_dict_func(i)
    all_cols = set()
    for k, v in funcs.items():
        all_cols = all_cols.union(( x[0] for x in v ))
    print('all_cols:', all_cols)

    for c in ['a', 'b']:
        print('-----', c, '-----')
        agg_funcs = funcs.get(0, funcs.get(c))
        if agg_funcs is None:
            continue
        agg_cols  = set([x[0] for x in agg_funcs])
        null_cols = all_cols - agg_cols 
        print('column',c)
        print('all ',all_cols)
        print('agg ',agg_cols)
        print('null ', null_cols)

        for n,f in agg_funcs:
            print(c, n,f)

=====
<function _create_function.<locals>._ at 0x7f649f8aed08>
all_cols: {'max'}
----- a -----
column a
all  {'max'}
agg  {'max'}
null  set()
a max <function _create_function.<locals>._ at 0x7f649f8aed08>
----- b -----
column b
all  {'max'}
agg  {'max'}
null  set()
b max <function _create_function.<locals>._ at 0x7f649f8aed08>
=====
max
all_cols: {'max'}
----- a -----
column a
all  {'max'}
agg  {'max'}
null  set()
a max <function _create_function.<locals>._ at 0x7f649f8aed08>
----- b -----
column b
all  {'max'}
agg  {'max'}
null  set()
b max <function _create_function.<locals>._ at 0x7f649f8aed08>
=====
('maxx', <function _create_function.<locals>._ at 0x7f649f8aed08>)
all_cols: {'maxx'}
----- a -----
column a
all  {'maxx'}
agg  {'maxx'}
null  set()
a maxx <function _create_function.<locals>._ at 0x7f649f8aed08>
----- b -----
column b
all  {'maxx'}
agg  {'maxx'}
null  set()
b maxx <function _create_function.<locals>._ at 0x7f649f8aed08>
=====
('maxx', 'max')
all_cols: {'maxx'}
----- a 

In [9]:
df.cols.groupby('id', 'name').agg({
    'fight':['sum', 'min', 'max'], 
    'trade':['max', 'count']}).data.grid()

AttributeError: 'NoneType' object has no attribute 'items'

In [28]:
pdf = df.data.grid()
help(pdf.agg)

Help on method aggregate in module pandas.core.frame:

aggregate(func, axis=0, *args, **kwargs) method of pandas.core.frame.DataFrame instance
    Aggregate using one or more operations over the specified axis.
    
    .. versionadded:: 0.20.0
    
    Parameters
    ----------
    func : function, str, list or dict
        Function to use for aggregating the data. If a function, must either
        work when passed a DataFrame or when passed to DataFrame.apply.
    
        Accepted combinations are:
    
        - function
        - string function name
        - list of functions and/or function names, e.g. ``[np.sum, 'mean']``
        - dict of axis labels -> functions, function names or list of such.
    axis : {0 or 'index', 1 or 'columns'}, default 0
            If 0 or 'index': apply function to each column.
            If 1 or 'columns': apply function to each row.
    *args
        Positional arguments to pass to `func`.
    **kwargs
        Keyword arguments to pass to `fun

In [7]:
# hash / rand columns which you wish to protect during ingest
df = (df
    .cols.find('greedy').rand()
    .cols.get('name').hashstr(salt='foobar')
    .rows.sample(3)
)

In [8]:
df.data.grid()

,id,name,trade,fight,greedy
0,0,D420C715,0.3,0.7,0.270087
1,1,FD200E54,0.6,0.4,0.703292
2,2,1ECC2478,0.2,0.8,0.848543
3,3,B0EBA258,0.8,0.2,0.246791


In [23]:
from pyspark.sql import functions as F
df.cols.agg({'type':'type', 'sample':'first'}).data.grid()

TypeError: 'str' object is not callable

INFO:kafka.client:Closing idle connection 1001, last active 17025801 ms ago
INFO:kafka.client:Closing idle connection 1001, last active 1688909 ms ago


In [16]:
df.save('races', 'minio')

True

In [16]:
dfc.list('minio', 'races').data.grid()

,name,type
0,_SUCCESS,FILE
1,part-00000-352df1b1-4682-44fd-b288-c8349233af4...,FILE
